In [2]:
import pandas as pd

train_df = pd.read_csv(r'C:\Users\85job\OneDrive\Desktop\datafolder\moviegenre\train_data.txt', sep=' ::: ', names=['Title', 'Genre', 'Description'], engine='python')
test_df = pd.read_csv(r'C:\Users\85job\OneDrive\Desktop\datafolder\moviegenre\test_data.txt', sep=' ::: ', names=['Title', 'Genre', 'Description'], engine='python')

In [3]:
print(train_df)

                                             Title        Genre  \
"1                    Oscar et la dame rose (2009)        drama   
2                                     Cupid (1997)     thriller   
"3                Young, Wild and Wonderful (1980)        adult   
"4                           The Secret Sin (1915)        drama   
"5                          The Unrecovered (2007)        drama   
...                                            ...          ...   
"54210                           ""Bonino"" (1953)       comedy   
"54211                 Dead Girls Don't Cry (????)       horror   
"54212   Ronald Goedemondt: Ze bestaan echt (2008)  documentary   
"54213                    Make Your Own Bed (1944)       comedy   
"54214  Nature's Fury: Storm of the Century (2006)      history   

                                              Description  
"1      Listening in to a conversation between his doc...  
2       A brother and sister with a past incestuous re...  
"3      As the 

In [4]:
print(test_df)

       Title                           Genre  \
0          1            Edgar's Lunch (1998)   
1          2        La guerra de papá (1977)   
2          3     Off the Beaten Track (2010)   
3          4          Meu Amigo Hindu (2015)   
4          5               Er nu zhai (1955)   
...      ...                             ...   
54195  54196  "Tales of Light & Dark" (2013)   
54196  54197     Der letzte Mohikaner (1965)   
54197  54198             Oliver Twink (2007)   
54198  54199               Slipstream (1973)   
54199  54200       Curitiba Zero Grau (2010)   

                                             Description  
0      L.R. Brane loves his life - his car, his apart...  
1      Spain, March 1964: Quico is a very naughty chi...  
2      One year in the life of Albin and his family o...  
3      His father has died, he hasn't spoken with his...  
4      Before he was known internationally as a marti...  
...                                                  ...  
54195  Cov

In [5]:
train_df.isnull()

,Title,Genre,Description
"""1",False,False,False
2,False,False,False
"""3",False,False,False
"""4",False,False,False
"""5",False,False,False
...,...,...,...
"""54210",False,False,False
"""54211",False,False,False
"""54212",False,False,False
"""54213",False,False,False


In [6]:
print(train_df['Genre'].value_counts())

drama          13613
documentary    13096
comedy          7447
short           5073
horror          2204
thriller        1591
action          1315
western         1032
reality-tv       884
family           784
adventure        775
music            731
romance          672
sci-fi           647
adult            590
crime            505
animation        498
sport            432
talk-show        391
fantasy          323
mystery          319
musical          277
biography        265
history          243
game-show        194
news             181
war              132
Name: Genre, dtype: int64


In [7]:
train_df = train_df.drop_duplicates()
test_df = test_df.drop_duplicates()

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)  
X_train = tfidf.fit_transform(train_df['Description']).toarray()
X_test = tfidf.transform(test_df['Description']).toarray()
y_train = train_df['Genre']
y_test = test_df['Genre']

C:\Users\85job\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_column = 'Genre' 

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()


X_train_vectorized = tfidf_vectorizer.fit_transform(train_df[text_column])
X_test_vectorized = tfidf_vectorizer.transform(test_df[text_column])


X_train = pd.DataFrame(X_train_vectorized.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
X_test = pd.DataFrame(X_test_vectorized.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Print shapes to verify
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: (54214, 30)
Test data shape: (54200, 30)


In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(class_weight='balanced',random_state = 42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [13]:
from sklearn.model_selection import train_test_split


X_small, _, y_small, _ = train_test_split(X_test, y_test, test_size=0.9, random_state=42)

In [14]:
from sklearn.metrics import classification_report

y_pred_small = model.predict(X_small)
print(y_small,y_pred_small) 

30257                         The 21st Second (2009)
35595                         "Yard Sale 411" (????)
22164           50 Ways to Leave Your Drummer (2015)
44594                         Ninja death III (1987)
4996                            The Runner (2016/VI)
                            ...                     
11284    In a Moment: The Johan Aspelin Story (????)
44732                           Unbreakable (2017/I)
38158                             It's Winter (2006)
860       The Willing Suspension of Disbelief (2007)
15795                    The Amazing Omelette (2005)
Name: Genre, Length: 5420, dtype: object ['war' 'war' 'war' ... 'war' 'war' 'war']


In [16]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores:", scores)
print("Average F1 score:", scores.mean())

Cross-validation F1 scores: [1. 1. 1. 1. 1.]
Average F1 score: 1.0
